# Upstream Regulator Analysis Package
## Arthritis Case Study

----------------------

Author: Mikayla Webster (m1webste@ucsd.edu)

Date: 19th January, 2018

----------------------

<a id='toc'></a>
## Table of Contents
1. [Background](#background)
2. [Import packages](#import)
3. [Load Networks](#load)
6. [P-value and Z-score Calculation](#pz)
7. [Compare the Ingenuity Article's results to Ours](#comp)
8. [Display Our results](#display)

## Background
<a id='background'></a>

Need some info about where these genes come from. Also Some feedback from Katie and Adam on motivation will go good here!

## Import packages
<a id='import'></a>

In [21]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import upstream regulator modules
import sys
code_path = '../ura'
sys.path.append(code_path)
import create_graph
import stat_analysis
reload(create_graph)
reload(stat_analysis)

# network visualization package
import visJS2jupyter.visJS_module as visJS_module # >> pip install visJS2jupyter

In [1]:
# User preferences
symbol = 'symbol'
entez = 'entrez'

gene_type = symbol

## Load Networks
<a id='load'></a>

1. List of all **Transcription Factors** (TF's) or regulators of interest to us
    <br>
    - Our sources are [slowkow](https://github.com/slowkow/tftargets) and [jaspar](http://jaspar.genereg.net/) TF databases
    <br><br>
2. **Background Network**: [STRING human protein interactions network](https://string-db.org/cgi/download.pl?UserId=9BGA8WkVMRl6&sessionId=HWUK6Dum9xC6&species_text=Homo+sapiens)  
    - Filter our background network down to just the sub network of TF's and their targets
    <br><br>
3. User-supplied list of **Differentially Expressed Genes** (DEG's)

In [2]:
# transcription factors
TF_list = create_graph.create_TF_list(slowkow_bool=True,
                    slowkow_files=['../TF_databases/slowkow_databases/TRED_TF.txt',
                                   '../TF_databases/slowkow_databases/ITFP_TF.txt',
                                   '../TF_databases/slowkow_databases/ENCODE_TF.txt',
                                   '../TF_databases/slowkow_databases/Neph2012_TF.txt',
                                   '../TF_databases/slowkow_databases/TRRUST_TF.txt',
                                   '../TF_databases/slowkow_databases/Marbach2016_TF.txt'],
                    slowkow_sep = '\n',
                    jaspar_bool=True,
                    jaspar_file="../TF_databases/jaspar_genereg_matrix.txt")
print "Number of TF's: " + str(len(TF_list))

Number of TF's: 3983


In [3]:
# background network
filename = "../background_networks/9606.protein.actions.v10.5.txt"
confidence_filter=400
DG_TF, DG_universe = create_graph.load_STRING_to_digraph(filename, confidence_filter, TF_list) # supplying TF_list filters network
print "Number of interactions: " + str(len(list(DG_TF.edges())))

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-7374...done.
Finished.
56 input query terms found dup hits:
	[(u'ENSP00000359550', 3), (u'ENSP00000447879', 2), (u'ENSP00000364076', 2), (u'ENSP00000348986', 2),
312 input query terms found no hit:
	[u'ENSP00000376684', u'ENSP00000289352', u'ENSP00000202788', u'ENSP00000373637', u'ENSP00000367802',
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
Number of interactions: 28939


In [28]:
def create_DEG_full_graph(filename):

    df = pd.DataFrame.from_csv(filename, sep='\t')

    # remove duplicate lines for same gene symbol, just use first occurance
    df.drop_duplicates(subset=['gene_symbol'], keep='first', inplace=True)

    DEG_full_list = df['gene_symbol']
    
    DEG_full_graph = nx.DiGraph()
    DEG_full_graph.add_nodes_from(DEG_full_list)

    return DEG_full_graph

def map_TF_to_DEG_attributes(filename, TF_list):
    
    DEG_list, DEG_to_adj_pvalue, DEG_to_updown = create_graph.create_DEG_list(DEG_filename, p_value_filter = 1)
    TF_to_adjp = { k: DEG_to_adj_pvalue[k] if k in DEG_to_adj_pvalue else 0 for k in TF_list }
    TF_to_foldchange = { k: DEG_to_adj_pvalue[k] if k in DEG_to_adj_pvalue else 0 for k in TF_list }
    
    return TF_to_adjp, TF_to_foldchange

In [29]:
# differentially expressed genes
DEG_filename = "../DEG_databases/DE_Coeff_OAvsNormal_OAvsNormal_20171215.csv" 
DEG_full_graph = create_DEG_full_graph(DEG_filename) # create graph of full DEG file
DEG_list, DEG_to_adj_pvalue, DEG_to_updown = create_graph.create_DEG_list(DEG_filename, 
                                                                          p_value_filter = 0.05, # p < 0.05
                                                                          fold_change_filter = 1) # |fld| > 1
DG_TF = create_graph.add_updown_from_DEG(DG_TF, DEG_to_updown) # add DEG information to our background network
print "Number of DEG's: " + str(len(DEG_list))

Number of DEG's: 1456


## P-values and Z-score Calculation
<a id='pz'></a>

1. **P-values**: How relevant is a TF to its DEG targets? Are they connected by chance, or is their connection statistically significant?
    <br>
    1. -log(p-value) for each TF using [scipy.stats.hypergeom.logsf](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.hypergeom.html).
        1. high value = significant connection between this TF and its DEG targets
        2. low value = TF is randomly associated with its DEG targets
        3. zero = None of this TF's targets were DEG's
        4. inf = original p-value was so small that its log is inf. Very high significance.
        <br><br>
2. **Z-scores**: Goal is to predict the activation states of the TF's

    - activation states: interaction type/regulation direction = predicted state
        - activating/up  = activating
        - activating/down = inhibiting
        - inhibiting/up = inhibiting
        - inhibiting/down = activating
        <br><br>
    - unbiased vs biased calculations:
        - **unbiased calculation**: Assume a normal distribution of activating and inhibiting states 
        - **biased calculation**: For the case when you cannot assume a 50-50 split between up/down-regulated targets and activating/inhbiting interactions. Modify our formula to approximate a normal distribution.

In [31]:
p_values_DEG_uni = stat_analysis.tf_pvalues(DG_TF, DEG_full_graph, DEG_list) # with respect to DEG universe

In [26]:
p_values_STRING_uni = stat_analysis.tf_pvalues(DG_TF, DG_universe, DEG_list) #

In [51]:
p_values_STRING_uni[50:70]

WDR3            inf
OLIG1           inf
ZNF281          inf
TRIM3           inf
ZBTB4           inf
ZNF24           inf
KIN             inf
ARNT      22.376288
TGFB1     21.262197
TP53      18.989094
HIF1A     17.696511
PPARG     14.845413
HMGB1     13.595826
FOSL1     13.495069
BRD2      12.435065
EP300     12.037722
SIRT1     11.840031
ARNTL     11.381461
RBPJ      10.491796
NRG1      10.480185
dtype: float64

In [52]:
p_values_DEG_uni[50:70]

ZNF346           inf
SIVA1            inf
MLX              inf
NAB1             inf
FBXO32           inf
IGF2BP3          inf
MAFB             inf
ARNT       18.988004
TGFB1      15.557520
HIF1A      13.596403
HMGB1      11.648640
TP53       11.595237
FOSL1      11.449493
PPARG      11.401583
BRD2       10.539068
ARNTL       9.634253
TAF1        9.123597
NRG1        9.076989
SIRT1       9.056385
EP300       8.884415
dtype: float64

In [19]:
z_scores = stat_analysis.tf_zscore(DG_TF, DEG_list, bias_filter = 0.25) # recommended bias filter is 0.25

## Display Our Results
- Display TF's with top z-scores
- Display where certain TF's rank among others and overall according to z-score
- Display subnetwork of a particular TF and its targets

In [68]:
def top_values(series, TF_to_adjp, TF_to_foldchange, act = True, abs_value = False, top = 10):

    # top activating and inhibiting, sort by strongest zscore or log(pvalue)
    if abs_value == True:
        top_series_abs = series.abs().sort_values(ascending=False).head(top)
        top_genes = list(top_series_abs.index)
        top_values = [series[gene] for gene in top_genes]
        return pd.Series(top_values, index=top_genes)

    # top activating
    if act == True:
        return series.sort_values(ascending=False).head(top)

    # top inhibiting
    else:
        return series.sort_values(ascending=True).head(top)
    
#df1['e'] = p.Series(np.random.randn(sLength), index=df1.index)
top_values(z_scores, TF_to_adjp, TF_to_foldchange, act = False, abs_value = True, top = 20)

<type 'list'>


RBL2     -3.162278
STAT1     2.840188
GLI2     -2.828427
FOXO3    -2.710687
KNTC1     2.672612
CLIP1     2.672612
CLASP1    2.672612
CKAP5     2.672612
CENPA     2.672612
MAD1L1    2.672612
CLASP2    2.672612
AHCTF1    2.672612
BUB3      2.672612
NCOR1     2.645751
ARNTL2    2.645751
STAT5A    2.523573
AKT2      2.500000
FOXM1     2.500000
STAT5B    2.500000
RORA     -2.449490
dtype: float64

In [81]:
series = z_scores.sort_values(ascending=False).head(10)
top_to_adjp = pd.Series({ k: TF_to_adjp[k] for k in TF_to_adjp if k in series })
top_to_foldchange = pd.Series({ k: TF_to_foldchange[k] for k in TF_to_foldchange if k in series })

In [86]:
pd.concat([series, top_to_adjp, top_to_foldchange], axis=1, names = ["z-scores", "adj p-value", "fold change"])

,0,1,2
AHCTF1,2.672612,0.043979,0.043979
BUB3,2.672612,0.045374,0.045374
CENPA,2.672612,0.000000,0.000000
CKAP5,2.672612,0.866232,0.866232
CLASP1,2.672612,0.002800,0.002800
CLASP2,2.672612,0.024884,0.024884
CLIP1,2.672612,0.042270,0.042270
KNTC1,2.672612,0.629780,0.629780
MAD1L1,2.672612,0.039497,0.039497
STAT1,2.840188,0.826469,0.826469


In [20]:
# Top TF's
# ~~~~~~~~~~~~~~~~~~~~~~ KATIE'S REQUEST ~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
TF_to_adjp, TF_to_foldchange = map_TF_to_DEG_attributes(DEG_filename, TF_list)
top_overall = stat_analysis.top_values(z_scores, act = False, abs_value = True, top = 20)
display(top_overall.to_frame(name = 'top z-scores'))

,top z-scores
RBL2,-3.162278
STAT1,2.840188
GLI2,-2.828427
FOXO3,-2.710687
KNTC1,2.672612
CLIP1,2.672612
CLASP1,2.672612
CKAP5,2.672612
CENPA,2.672612
MAD1L1,2.672612


In [25]:
# Ranking (in this case from 0 to 168)
genes_to_rank = ['HOXA4', 'CEBPB', 'HIF1A','KLF4', 'TLE3', 'RBL2','TP53', 'STAT1', 'STAT5B']
stat_analysis.rank_and_score_df(z_scores, genes_to_rank, act = True, abs_value = True, remove_dups=True)

,rank,z-score
RBL2,0,-3.162278
STAT1,1,2.840188
STAT5B,7,2.500000
CEBPB,29,-1.632993
HIF1A,37,-1.333333
TLE3,46,-1.000000
KLF4,49,-0.832050
TP53,65,0.321634
HOXA4,70,0.000000


In [27]:
stat_analysis.vis_tf_network(DG_TF, 'KLF5', '../DEG_databases/geo2r_GSE11352_brca_48hours.txt', DEG_list,
              directed_edges = True,
              node_spacing = 1500,
              graph_id = 2) 

In [23]:
# display subnetworks using visJS2jupyter
stat_analysis.vis_tf_network(DG_TF, 'STAT1', '../DEG_databases/geo2r_GSE11352_brca_48hours.txt', DEG_list,
              directed_edges = True,
              node_spacing = 2000,
              graph_id = 1) 